In [ ]:
# time series prediction
# two tasks:
# predict tomorrow's price
# predict next week's price

In [1]:
# imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [3]:
# load data
df = pd.read_csv('data/gas_price_2017_2023.csv')

In [4]:
df

,Date,VANCOUVER
0,1/2/2017,130.6
1,1/3/2017,135.8
2,1/4/2017,135.9
3,1/5/2017,134.1
4,1/6/2017,135.5
...,...,...
1823,12/26/2023,NaN
1824,12/27/2023,NaN
1825,12/28/2023,NaN
1826,12/29/2023,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1828 entries, 0 to 1827
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1828 non-null   object 
 1   VANCOUVER  1711 non-null   float64
dtypes: float64(1), object(1)
memory usage: 28.7+ KB


In [8]:
# feature scaling
scaler = MinMaxScaler()
training_set_scaled = scaler.fit_transform(df['VANCOUVER'])
training_set_scaled

ValueError: Expected 2D array, got 1D array instead:
array=[130.6 135.8 135.9 ...   nan   nan   nan].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
# creating data with timesteps

In [1]:
# tomorrow's price prediction

In [2]:
# next week's price prediction